In [35]:
import json
import requests
from requests import Response
from pydantic import BaseModel,RootModel,Field,ValidationError,field_validator

class UB(BaseModel):
    場站中文名稱:str = Field(alias='sna')
    場站區域:str = Field(alias='sarea')
    資料更新時間:str = Field(alias='mday')
    地點:str = Field(alias='ar')
    全站禁用狀態:bool = Field(alias='act')
    場站總停車格:int = Field(alias='total')
    場站目前車輛數量:int = Field(alias='available_rent_bikes')
    空位數量:int = Field(alias='available_return_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')

class Root(RootModel):
    root:list[UB]

youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
res:Response = requests.get(youbike_url)
all_data = res.json()

try:
    root:Root = Root.model_validate(all_data)
except ValidationError as error:
    print(error)

data_list:list[dict] = root.model_dump()
filter_list:list[dict] = list(filter(lambda item:False if item['全站禁用狀態']==0 else True,data_list))
filter_list_rent_less3:list[dict] = list(filter(lambda item:True if item['場站目前車輛數量']<3 else False,data_list))
filter_list_return_less3:list[dict] = list(filter(lambda item:True if item['空位數量']<3 else False,data_list))

print(f"台北市YouBike有{len(data_list)}站點，目前正常運作站點共{len(filter_list)}個")
print(f"目前可借車輛數為3輛以內的站點有{len(filter_list_rent_less3)}站")
#print(filter_list_rent_less3)
print(f"目前可還車輛數為3輛以內的站點有{len(filter_list_return_less3)}站")
#print(filter_list_return_less3)

台北市YouBike有1420站點，目前正常運作站點共1411個
目前可借車輛數為3輛以內的站點有244站
目前可還車輛數為3輛以內的站點有97站


[{'場站中文名稱': 'YouBike2.0_公館公園',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-13 07:40:27',
  '地點': '羅斯福路四段113巷基隆路四段41巷口',
  '全站禁用狀態': True,
  '場站總停車格': 15,
  '場站目前車輛數量': 15,
  '空位數量': 0,
  '緯度': 25.01137,
  '經度': 121.54062},
 {'場站中文名稱': 'YouBike2.0_臺大動物醫院前',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-13 07:42:19',
  '地點': '基隆路三段155巷/基隆路四段口',
  '全站禁用狀態': True,
  '場站總停車格': 18,
  '場站目前車輛數量': 17,
  '空位數量': 1,
  '緯度': 25.01576,
  '經度': 121.54291},
 {'場站中文名稱': 'YouBike2.0_芳和國中',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-13 07:42:20',
  '地點': '臥龍街170號(前)',
  '全站禁用狀態': True,
  '場站總停車格': 37,
  '場站目前車輛數量': 36,
  '空位數量': 1,
  '緯度': 25.0189,
  '經度': 121.55013},
 {'場站中文名稱': 'YouBike2.0_捷運大安站(2號出口)',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-13 07:38:16',
  '地點': '信義路3段143號(前)',
  '全站禁用狀態': True,
  '場站總停車格': 22,
  '場站目前車輛數量': 21,
  '空位數量': 1,
  '緯度': 25.03358,
  '經度': 121.54141},
 {'場站中文名稱': 'YouBike2.0_仁愛國中',
  '場站區域': '大安區',
  '資料更新時間': '2024-06-13 07:41:21',
  '地點': '安和路一段75號(對面)',
  '全站禁用狀態': True,
  '場站總停車格': 25,
 